**Please write the names of all group members here:** Giulio Vittorio Carassai, Sören Lambrecht, Alihan Kerestecioglu




---


*Note:* The provided structure for the code below is only suggestive, and if you want to structure your programs differently you may do so.

Exercise 1. (Write down the general equation for $V$ depending on the given parameters. You may use the Black-Scholes formulas without proof.)


we first define for each underlying asset j = 1, ..., 4 the following quantities:

$$ d_1^j = \frac{\log (S_0^j/K) + (r + \sigma^2/2)T}{\sigma \sqrt{t}} $$

$$ d_2^j = d_1^j - \sigma \sqrt{T} $$

Applying Black-Scholes Call Option Price we then define for j = 1,2 (in the following we use the notation where $\mathcal{N}(\cdot)$ is the CDF of a std gaussian):

$$ V_j = S_0^j \mathcal{N}(d_1^j) - K e^{-rT} \mathcal{N}(d_2^j) \;\;\;\;\;\;\;\;\;\;\;\; j = 1,2$$

and for j = 3,4 we define the Put Option Price similarly as:

$$ V_j = K e^{-rT} N(-d_2^j) - S_0^j N(- d_1^j) \;\;\;\;\;\;\;\;\;\;\;\; j = 3,4$$

It then follows that the whole portfolio time 0 value is 

$$ V = \sum_{j=1}^4 V_j$$

In [50]:
# Exercise 2.a)
# supressing warnings of deprecated versions of methods
import warnings
warnings.filterwarnings("ignore")

# For all exercises in this assignment, the following packages are sufficient:
import numpy as np
from scipy.stats import norm  # standard normal distribution

# You may use the black scholes formulas from below
def black_scholes_call(s0, K, T, r, sigma):
    d1 = (np.log(s0/K) + (r + sigma**2/2)*T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return s0 * norm.cdf(d1) - K * np.exp(-r*T)* norm.cdf(d2)

def black_scholes_put(s0, K, T, r, sigma):
    return black_scholes_call(s0, K, T, r, sigma) - s0 + np.exp(-T * r) * K


# Implement the function V
# As a sanity check:  For the given parameters, the value for
#                     x = (100, 100, 100, 100) should be around 55
def V(x,K=100,T=1,r=0.02, sigmas=[0.1*(1+i) for i in range(1,5)]):
    # call values
    value_calls = 0
    for i in range(0,2):
        value_calls += black_scholes_call(x[:,i], K, T, r, sigmas[i])
    # put values
    value_puts = 0
    for i in range(2,4):
        value_puts += black_scholes_put(x[:,i], K, T, r, sigmas[i])
    #return sum of all values
    return value_calls + value_puts

print("Exercise 2.a)")
print("Sanity Check: ", V(np.asarray([[100, 100, 100, 100]]))[0], "is almost 55")

# Simulate (x_1, ..., x_1000) and (y_1, ..., y_1000)
np.random.seed(0)  # fix the random seed so the results are reproducible
m = 1000
x_training = np.random.uniform(low=80., high=120., size=[m, 4])
y = V(x_training)  # or similar

# Calculate the matrix A
A = np.concatenate((np.ones((m,1)), x_training), axis=1)

# Solve the normal equation; you may for instance use np.linalg.solve
b = np.linalg.solve(np.transpose(A) @ A, np.transpose(A) @ y)

# (i) Report the approximation error for x = (100, 100, 100, 100)
def f_linear(x, beta = b):
    return beta[0] + x @ beta[1:5]

x = 100*np.ones((1,4))
print("i) Approximation Error for (100,100,100,100) = ", (f_linear(x)-V(x))[0])

# (ii) Report the L^1 approximation error.
# Generate n test samples, calculate the confidence interval, and
# potentially readjust the value of n if the bounds are too wide.
n = 100
x_test = np.random.uniform(low=80., high=120., size=[n, 4])

def l1_error_sample(x_test, b):
    return np.abs(f_linear(x_test, b) - V(x_test))

l1_error_approx_sample = l1_error_sample(x_test, b)

# find CI
import scipy.stats

def get_conf_int(l1_error_approx_sample):
    m, std = np.mean(l1_error_approx_sample), np.std(l1_error_approx_sample)
    h = std/np.sqrt(len(l1_error_approx_sample)) * scipy.stats.norm.ppf(0.975)
    return (m-h, m+h)

print("ii) for n = ", n)
print("L1 Approximation Error:", np.mean(l1_error_approx_sample))
print("CI: ", get_conf_int(l1_error_approx_sample))

Exercise 2.a)
Sanity Check:  55.03296129776206 is almost 55
i) Approximation Error for (100,100,100,100) =  3.2782988222492975
ii) for n =  100
L1 Approximation Error: 1.1446840655196078
CI:  (0.9804158201775574, 1.308952310861658)


In [42]:
# Exercise 2.b)
# Calculate the matrix B. The function np.concatenate may be useful
def create_B(data):
    length = data.shape[0]
    B = np.ones(length).reshape(length,1)
    B = np.concatenate((B, data), axis=1)
    for i in range(4):
        for j in range(i,4):
            val = data[:, j] * data[:, i]
            B = np.concatenate((B, val.reshape(length,1)), axis = 1)
    return B

B = create_B(x_training)

# Solve the normal equation
b_poly = np.linalg.solve(B.T @ B, B.T @ y)
b_poly

def f_poly(x, beta):
    return x @ beta

# (i) Report the approximation error for x = (100, 100, 100, 100)
x = 100*np.ones((1,4))
x_transformed = create_B(x)
print("Exercise 2.b)")
print("i) Approximation Error for (100,100,100,100) = ", (f_poly(x_transformed, b_poly) - V(x))[0])

# (ii) Report the L^1 approximation error.
# Generate n test samples, calculate the confidence interval, and
# potentially readjust the value of n if the bounds are too wide.

n = 3000
x_test = np.random.uniform(low=80., high=120., size=[n, 4])


x_test_transformed = create_B(x_test)
l1_error_approx = np.abs(f_poly(x_test_transformed, b_poly) - V(x_test))

#do the CI
print("ii) We chose n = ", n, " so that the width of the CI is less then the approximation error")
print("L1 Approximation Error:", np.mean(l1_error_approx))
print("CI: ", get_conf_int(l1_error_approx))


Exercise 2.b)
i) Approximation Error for (100,100,100,100) =  0.018710719169050094
ii) We chose n =  3000  so that the width of the CI is less then the approximation error
L1 Approximation Error: 0.06894113653448584
CI:  (0.06712154944284648, 0.0707607236261252)


In [43]:
# Exercise 3. a)
# Define a general function Vmc taking as input x = s_0
# (it helps to implement this with a variable number of Monte Carlo simulations)

# set varibales apart from s0
K = 100
T = 1
r = 0.02
sigma = np.array([(j + 1)*0.1 for j in range(1,5)])

def V_mc(s0, n_iterations):
    payoff = []
    for _ in range(n_iterations):
        W = np.random.randn(4) # mean 0 std dev 1
        s = s0 * np.exp((r - 0.5*sigma**2) * T + sigma*W)
        max_s = np.max(s)
        
        payoff.append(np.maximum((max_s - K), 0))
        
    return np.exp(-r * T) * np.mean(payoff)

# Evaluate Vmc at x = (100, 100, 100, 100)
# Sanity check: While the value depends on the particular Monte-Carlo sample, it should be around 40 to 45

n_iterations = 10000
s0 = np.array([100, 100, 100, 100])

print("Exercise 3. a)")
print("Monte Carlo Estimate:", V_mc(s0, n_iterations))

Exercise 3. a)
Monte Carlo Estimate: 42.58197407478139


In [49]:
# Exercise 3.b)
# Simulate points x_i and y_i as described.

K = 100
T = 1
r = 0.02
sigma = np.array([(j + 1)*0.1 for j in range(1,5)])

n = 1000
np.random.seed(0)
x_simulated = np.random.uniform(low=80., high=120., size=[n, 4])
# print('x_simulated shape:', x_simulated.shape)

# Note that y_i are basically evaluations of Vmc using only 1 simulation to
# compute the Monte Carlo average
xi = np.random.randn(n, 4)

# Calculate the max call option payoffs
ST = x_simulated * np.exp(sigma * xi - 0.5 * sigma**2 * T) - K * np.exp(-r*T)
# print('ST shape:', ST.shape)

y_simulated = np.maximum(np.max(ST, axis = 1), 0)
# print('sigma shape:', sigma.shape)
# print('y_simulated shape:', y_simulated.shape)

# Compute normal least square regression by solving the normal equation

B = create_B(x_simulated)
# Solve the normal equation
b_poly_2 = np.linalg.solve(B.T @ B, B.T @ y_simulated)
# print('Shape of Extended Design matrix:', B.shape)
# print('Shape of Parameters:', b_poly_2.shape)

def g(x, beta):
    return x @ beta

print("Exercise 3.b)")
print('Estimated parameters of function g(.) :', b_poly_2)  # small check

#report approximation error on x = (100,100,100,100)
s0 = np.array([100,100,100,100]).reshape(1,4)
approximation_error = g(create_B(s0), b_poly_2)[0] - V_mc(s0, 10000)
print('-----------------------------------------------')
print('Approximation error for x = (100,100,100,100): ', approximation_error)

Exercise 3.b)
Estimated parameters of function g(.) : [ 6.16544863e+02 -3.64108819e+00 -7.29507070e+00 -1.92737812e+00
 -1.53385319e-01  3.15508813e-02 -8.23239822e-03 -5.01863464e-03
 -1.33919975e-02  3.56292318e-02  5.61392416e-03  7.62763392e-03
  7.60933558e-03  8.05530013e-03  2.77190035e-03]
-----------------------------------------------
Approximation error for x = (100,100,100,100):  -5.610133960613133


In [51]:
# Exercise 3.c) 
print("Exercise 3.c)")
n = 1000
np.random.seed(0)
x_simulated = np.random.uniform(low=80., high=120., size=[n, 4])
xi = np.random.randn(n, 4)
ST = x_simulated * np.exp(sigma * xi - 0.5 * sigma**2 * T) - K * np.exp(-r*T)
y = np.maximum(np.max(ST, axis = 1), 0) # simulated y values
X = create_B(x_simulated) # extended design matrix

# To calculate the singular value decomposition you can use np.linalg.svd

# First split the data into 5 subgroups
# (while this is inefficient for memory, for this small example here it's fine)

# Define a plausible truncation threshold c > 0

def get_pseudo_in(B, threshold):
    U,S,Vh = np.linalg.svd(B, full_matrices=False)
    S_trunc = np.where(S > threshold, 1/S, 0)
    # Compute the truncated pseudoinverse of B
    B_pseudo_inv = Vh.T @ np.diag(S_trunc) @ U.T
    return B_pseudo_inv


from collections import defaultdict
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

#eigenvalues, _ = np.linalg.eig(np.dot(X.T, X))
#print(np.sqrt(np.max(eigenvalues)) * 0.0001)
thresholds = [0.5, 1, 10, 100, 1000, 10000, 100000]
cv_scores = defaultdict(list) # format will be like {0.1: [f1_s, f2_s], 0.2: [f1_s, f2_s]}

# Iterate over the subgroups (each subgroup serves as validation data once)
# START ITERATE HERE
for i, (train_index, val_index) in enumerate(kf.split(X)):
    #print(f"Fold {i}")
    #print(f"  Validation:  index={val_index[0]} to {val_index[-1]}")
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    for threshold in thresholds:
        # fit
        X_inv = scipy.linalg.pinv(X_train, rcond=threshold)
        #X_inv = get_pseudo_in(X_train, threshold=threshold)
        beta = np.dot(X_inv, y_train)
        # predict val set
        y_pred = X_val @ beta
        # compute error
        err = np.mean((y_pred - y_val)**2)  #np.mean(np.abs(y_pred - y_val))
        cv_scores[threshold].append(err)

#print('------------')       
#print(cv_scores)
#print('------------')        

# END ITERATE HERE
# Calculate the generalization error as the average over the different
# approximation errors across the 5 subgroups.
threshold_avg = {}
for threshold, scores in cv_scores.items():
    threshold_avg[threshold] = np.mean(scores)
    print(f'For threshold {threshold}: achieved an approximation error of {np.mean(scores)}')

# Try a few different truncation levels and choose one which achieves good
# generalization error. You can comment for clarity all values you tried

# Report the approximation error
import operator
best_threshold = min(threshold_avg.items(), key=operator.itemgetter(1))[0]
print(f'Achieved best average CV score of {threshold_avg[best_threshold]}, with the threshold {best_threshold}') 

#report approximation error on x = (100,100,100,100)
s0 = np.array([100,100,100,100]).reshape(1,4)

#fit model with best threshhold
X_inv_all = scipy.linalg.pinv(X, rcond= best_threshold)
beta_all = np.dot(X_inv_all, y)

g_x =  create_B(s0) @ beta_all

approximation_error = g_x - V_mc(s0, 10000)
print('------------------------------------------------')
print('Approximation error for x = (100,100,100,100) : ', approximation_error[0])

Exercise 3.c)
For threshold 0.5: achieved an approximation error of 1916.1573299922443
For threshold 1: achieved an approximation error of 1916.1573299922443
For threshold 10: achieved an approximation error of 1916.1573299922443
For threshold 100: achieved an approximation error of 1907.813056067831
For threshold 1000: achieved an approximation error of 1917.7301156937642
For threshold 10000: achieved an approximation error of 1903.67936490263
For threshold 100000: achieved an approximation error of 1909.8049959856246
Achieved best average CV score of 1903.67936490263, with the threshold 10000
------------------------------------------------
Approximation error for x = (100,100,100,100) :  4.895709754677959


In [46]:
# Exercise 3. d) 
print("Exercise 3. d)")
#For each of the approximating functions g obtained in b)–d), report the approximation error g(x) − Vˆmc(x) 
#for x = (100, 100, 100, 100), where Vˆmc(x) is the Monte Carlo estimate from a).
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import pandas as pd
param_grid = {
    'alpha'  : [0.01, 1, 10, 100, 1000, 10000, 11000]
}
estimator = Ridge()
reg = GridSearchCV(estimator, param_grid, scoring= "neg_mean_squared_error", cv=5) #custom_scorer
reg.fit(X, y)
cv_scores = pd.DataFrame(reg.cv_results_)[[f'split{i}_test_score' for i in range(5)] + ['mean_test_score']]
#print(cv_scores.head(10))
print(f'Achieved best average CV score of {-reg.best_score_}, with {reg.best_estimator_}') 

#report approximation error on x = (100,100,100,100)
s0 = np.array([100,100,100,100]).reshape(1,4)

#fit model with best threshhold
g_x = reg.predict(create_B(s0))

approximation_error = g_x - V_mc(s0, 10000)
print('------------------------------------------------')
print('Approximation error for x = (100,100,100,100) : ', approximation_error[0])

Exercise 3. d)
Achieved best average CV score of 1907.5530614113218, with Ridge(alpha=1000)
------------------------------------------------
Approximation error for x = (100,100,100,100) :  -2.5722855066988686


In [47]:
# Exercise 4 a) 
print("Exercise 4 a)")
# Simulate new labels y_i^L. Note that those correspond to Vmc evaluations with
# L many simulations for the Monte Carlo estimate

K = 100
T = 1
r = 0.02
sigma = np.array([(j + 1)*0.1 for j in range(1,5)])

L = 1000
n = 1000

x_simulated = np.random.uniform(low=80., high=120., size=[n, 4])
#print('x_simulated shape:', x_simulated.shape)

y_simulated_reduced = np.zeros(n)
for l in range(L):

    # Note that y_i are basically evaluations of Vmc using only 1 simulation to
    # compute the Monte Carlo average
    xi = np.random.randn(n, 4)

    # Calculate the max call option payoffs
    ST = x_simulated * np.exp(sigma * xi - 0.5 * sigma**2 * T) - K * np.exp(-r*T)
    #print('ST shape:', ST.shape)

    y_simulated = np.maximum(np.max(ST, axis = 1), 0)
    #print('sigma shape:', sigma.shape)
    #print('y_simulated shape:', y_simulated.shape)
    
    y_simulated_reduced += y_simulated
    
y_simulated_reduced = y_simulated_reduced / L

#print('y_simulated_reduced shape:', y_simulated_reduced.shape)

# a)
# solve the least square regression by solving the normal equation
x_transformed = create_B(x_simulated)
# Solve the normal equation
b_poly_reduced = np.linalg.solve(B.T @ B, B.T @ y_simulated_reduced)
# print('Shape of Extended Design matrix:', B.shape)
# print('Shape of Parameters:', b_poly_reduced.shape)

def g(x, beta):
    return x @ beta

print('Parameters Estimated:', b_poly_reduced)
# print('-------------------------------------')
# print('Average training approximation error:',np.mean((g(x_transformed, b_poly_reduced) - y_simulated_reduced )))

Exercise 4 a)
Parameters Estimated: [ 1.87507056e+02 -4.19151790e-01 -4.32126314e-01 -6.27054486e-01
 -1.39228704e+00  2.31845707e-03 -1.55180749e-03  3.00824625e-04
  8.44416254e-04  4.25226078e-04  2.12954533e-04  5.19065521e-03
  1.95195477e-03  1.81011660e-03  2.90728954e-03]


In [48]:
# Exercise 4b)
print("Exercise 4b)")
# Solve the problem using truncated pseudoinversion and/or ridge regression as
# in Exercise 3 c) or d).
# What are the levels of optimal regularization and truncation?
# (How do they differ to the ones in Exercise 3?)

kf = KFold(n_splits=5)

#eigenvalues, _ = np.linalg.eig(np.dot(X.T, X))
#print(np.sqrt(np.max(eigenvalues)) * 0.0001)
thresholds = [0.0001, 0.001, 0.01, 0.05, 0.1, 0.15, 0.25, 1]
cv_scores = defaultdict(list) # format will be like {0.1: [f1_s, f2_s], 0.2: [f1_s, f2_s]}

# Iterate over the subgroups (each subgroup serves as validation data once)
# START ITERATE HERE
for i, (train_index, val_index) in enumerate(kf.split(x_transformed)):
    #print(f"Fold {i}")
    #print(f"  Validation:  index={val_index[0]} to {val_index[-1]}")
    X_train, X_val = X[train_index], x_transformed[val_index]
    y_train, y_val = y[train_index], y_simulated_reduced[val_index]
    for threshold in thresholds:
        # fit
        X_inv = scipy.linalg.pinv(X_train, rcond=threshold)
        #X_inv = get_pseudo_in(X_train, threshold=threshold)
        beta = np.dot(X_inv, y_train)
        # predict val set
        y_pred = X_val @ beta
        # compute error
        err = np.mean((y_pred - y_val)**2) 
        cv_scores[threshold].append(err)

#print('------------')       
#print(cv_scores)
#print('------------')        

# END ITERATE HERE
# Calculate the generalization error as the average over the different
# approximation errors across the 5 subgroups.
threshold_avg = {}
for threshold, scores in cv_scores.items():
    threshold_avg[threshold] = np.mean(scores)
    print(f'For threshold {threshold}: achieved an approximation error of {np.mean(scores)}')

# Try a few different truncation levels and choose one which achieves good
# generalization error. You can comment for clarity all values you tried

# Report the approximation error
best_threshold = min(threshold_avg.items(), key=operator.itemgetter(1))[0]
print('-----------------------------------------------')
print(f'With Pseudo Inverse, achieved best average CV score of {threshold_avg[best_threshold]}, with the threshold {best_threshold}')
print('-----------------------------------------------')

param_grid = {
    'alpha'  : [1, 10, 100, 1000, 1100, 1200, 10000]
}
estimator = Ridge()
reg = GridSearchCV(estimator, param_grid, scoring='neg_mean_squared_error', cv=5)
reg.fit(x_transformed, y_simulated_reduced)
cv_scores = pd.DataFrame(reg.cv_results_)[[f'split{i}_test_score' for i in range(5)] + ['mean_test_score']]
#print(cv_scores.head(10))
print(f'With Ridge, achieved best average CV score of {-reg.best_score_}, with {reg.best_estimator_}')

#generate some test data to make a comparision
s0 = np.random.uniform(low=80., high=120., size=[100, 4])

#print('x_simulated shape:', x_simulated.shape)

V_mc_reduced = np.zeros(100)
for l in range(1000):

    # Note that y_i are basically evaluations of Vmc using only 1 simulation to
    # compute the Monte Carlo average
    xi = np.random.randn(100, 4)

    # Calculate the max call option payoffs
    ST = s0 * np.exp(sigma * xi - 0.5 * sigma**2 * T) - K * np.exp(-r*T)
    #print('ST shape:', ST.shape)

    y_simulated = np.maximum(np.max(ST, axis = 1), 0)
    #print('sigma shape:', sigma.shape)
    #print('y_simulated shape:', y_simulated.shape)
    
    V_mc_reduced += y_simulated
    
V_mc_reduced = V_mc_reduced / 1000

#ridge
g_x = reg.predict(create_B(s0))
abs_approximation_error_ridge = np.mean(np.abs(g_x - V_mc_reduced))

#linear reg
abs_approximation_error_lin = np.mean(np.abs(g(create_B(s0), b_poly_reduced)- V_mc_reduced))

print(f'With Ridge, the abs approximation error is {abs_approximation_error_ridge}, \n and with linear regression \ it is {abs_approximation_error_lin}')

Exercise 4b)
For threshold 0.0001: achieved an approximation error of 61.60660033763121
For threshold 0.001: achieved an approximation error of 61.60660033763121
For threshold 0.01: achieved an approximation error of 61.60660033763121
For threshold 0.05: achieved an approximation error of 61.60660033763121
For threshold 0.1: achieved an approximation error of 61.60660033763121
For threshold 0.15: achieved an approximation error of 53.51796047965136
For threshold 0.25: achieved an approximation error of 53.51796047965136
For threshold 1: achieved an approximation error of 53.51796047965136
-----------------------------------------------
With Pseudo Inverse, achieved best average CV score of 53.51796047965136, with the threshold 0.15
-----------------------------------------------
With Ridge, achieved best average CV score of 1.8078501995901164, with Ridge(alpha=100)
With Ridge, the abs approximation error is 1.0975065882756587, 
 and with linear regression \ it is 6.2195834708630215


We validated truncated pseudoinverse and ridge regression regularizations (with the optimal hyperparameters found in previous cross-validations) and found that ridge regression performs superior to the pseudo inverse.

When applying ridge and the plain linear regression from a) on a made up test set of 100 data points. We can see that ridge is better at predicting which we can interpret as a better variance bias trade-off. 